In [1]:
import os
import argparse
import pprint
from datetime import date
import _init_paths

import numpy as np
import random
import warnings
import torch
import torch.nn as nn
from config import config, update_config
import create_logger
from lib import metric_clip as metric


from core.model_prompt import model
from dataset.iterator_factory import get_dataiter
from networks.get_symbol import get_symbol
from lib import criterions


torch.backends.cudnn.enabled = False

/home/qchenax/miniconda3/envs/astar/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [2]:
update_config('./exps/fed_non_iid_v2.yaml')
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

if config.seed:
    random.seed(config.seed)
    np.random.seed(config.seed)
    torch.manual_seed(config.seed)
    torch.cuda.manual_seed(config.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    warnings.warn('You have chosen to seed training. '
                    'This will turn on the CUDNN deterministic setting, '
                    'which can slow down your training considerably! '
                    'You may see unexpected behavior when restarting '
                    'from checkpoints.')

data_loader, data_iter = get_dataiter('data_name', config, 'fed')

pmpt_1 = data_iter.pmpt_1 if hasattr(data_iter, 'pmpt_1') else None
sym_net = get_symbol(config.net.name, config, hand_craft=config.net.hand_craft, pmpt_1 = pmpt_1)
sym_net.float()

# model_prefix = os.path.join(config.output_pt, 'exp_'+ model_fixtime + '_' + config.net.name)
model_prefix = 'exp_' + config.net.name

criterion = criterions.KLLoss_fast()

net = model(net=sym_net, criterion=criterion, model_prefix=model_prefix, step_callback_freq=config.train.callback_freq,
            save_checkpoint_freq=config.save_frequency, logger = None)

/home/qchenax/astar/rul_prompt/config.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  exp_config = edict(yaml.load(f))
/tmp/ipykernel_2073/1597031922.py:11: UserWarning: You have chosen to seed training. This will turn on the CUDNN deterministic setting, which can slow down your training considerably! You may see unexpected behavior when restarting from checkpoints.
  warnings.warn('You have chosen to seed training. '


Sensor::test data (23) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (22) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (21) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (29) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (25) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (29) less than seq_len (30)!
Sensor::Use first data to pad!
Setting::test data (23) less than seq_len (30)!
Setting::Use first data to pad!
Setting::test data (22) less than seq_len (30)!
Setting::Use first data to pad!
Setting::test data (21) less than seq_len (30)!
Setting::Use first data to pad!
Setting::test data (29) less than seq_len (30)!
Setting::Use first data to pad!
Setting::test data (25) less than seq_len (30)!
Setting::Use first data to pad!
Setting::test data (29) less than seq_len (30)!
Setting::Use first data to pad!
Sensor::test data (23) less than seq_len (30)!
Senso

In [6]:
for k in net.net.state_dict().keys():
    if k.startswith('fc'):
        print(k)

fc.fc1.weight
fc.fc1.bias
fc.fc2.weight
fc.fc2.bias
